In [4]:
from pandas import Series, Timestamp
import pandas as pd
from pandas import DataFrame
import numpy as np
import simplejson as json

In [5]:
def load_jsons(data_path, file):
    """ helper function to load '.json' files (they're not proper jsons) """
    file_path = data_path + file
    with open(file_path) as jsons:
        lines = [json.loads(json_line) for json_line in jsons]
    return pd.DataFrame(lines)

In [6]:
# Load data for town called Ambridge
data_path = './data/ambridge/'

# Seperate the files 
review_file = 'review.json'
business_file = 'business.json'
user_file = 'user.json'
tip_file = 'tip.json'
checkin_file = 'checkin.json'

reviews = load_jsons(data_path, review_file)
businesses = load_jsons(data_path, business_file)
users = load_jsons(data_path, user_file)
tips = load_jsons(data_path, tip_file)
checkins = load_jsons(data_path, checkin_file)

display(businesses)

FileNotFoundError: [Errno 2] No such file or directory: './data/ambridge/review.json'

In [7]:
"""
All these functions were taken and modified from module 3
"""

def extract_categories(businesses):
    categories = businesses.apply(lambda row: pd.Series([row['business_id']] + row['categories'].lower().split(",")), axis=1)
    stack_categories = categories.set_index(0).stack()
    df_stack_categories = stack_categories.to_frame()
    df_stack_categories['business_id'] = stack_categories.index.droplevel(1)
    df_stack_categories.columns = ['category', 'business_id']
    return df_stack_categories.reset_index()[['business_id', 'category']]

def pivot_categories(df):
    """Create a one-hot encoded matrix for categories.
    
    Arguments:
    df -- a dataFrame containing at least the columns 'businessId' and 'category'
    
    Output:
    a matrix containing '0' or '1' in each cell.
    1: the business has the category
    0: the business does not have the category
    """
    return df.pivot_table(index = 'business_id', columns = 'category', aggfunc = 'size', fill_value=0)

def create_similarity_matrix_jaccard(matrix):
    m11 = matrix @ matrix.T
    m10 = pd.DataFrame(matrix.sum(axis = 1).values + np.zeros(m11.shape), index = m11.index, columns = m11.index)
    m01 = m10.T
    return m11/(m01 + m10 - m11)

categories = extract_categories(businesses)
df_utility_categories = pivot_categories(categories)
similarity_matrix_categories = create_similarity_matrix_jaccard(df_utility_categories)

NameError: name 'businesses' is not defined

In [8]:
display(similarity_matrix_categories.head())

NameError: name 'similarity_matrix_categories' is not defined

In [9]:
user_dataframe = pd.DataFrame(users)

display(user_dataframe)

NameError: name 'users' is not defined

In [10]:
def predict_ratings(similarity, utility, to_predict):
    """Predicts the predicted rating for the input test data.
    
    Arguments:
    similarity -- a dataFrame that describes the similarity between items
    utility    -- a dataFrame that contains a rating for each user (columns) and each movie (rows). 
                  If a user did not rate an item the value np.nan is assumed. 
    to_predict -- A dataFrame containing at least the columns movieId and userId for which to do the predictions
    """
    # copy input (don't overwrite)
    ratings_test_c = to_predict.copy()
    # apply prediction to each row
    ratings_test_c['predicted category'] = to_predict.apply(lambda row: predict_ids(similarity, utility, row['categories'], row['business_id']), axis=1)
    return ratings_test_c


def predict_ids(similarity, utility, userId, itemId):
    # select right series from matrices and compute
    if userId in utility.columns and itemId in similarity.index:
        return predict_vectors(utility.loc[:,userId], similarity[itemId])
    return 0

In [11]:
prediction = predict_ratings(similarity_matrix_categories, businesses, businesses[['name', 'business_id', 'categories']])
display(prediction)

NameError: name 'similarity_matrix_categories' is not defined